Try to use this sales data with atleast very important and important options...
https://drive.google.com/file/d/1MZI4XIofL-0QpMIr9sFODSKVkexrSZ1A/view?usp=drive_link

In [0]:
sales_path="/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/sales.csv"
df_sales=spark.read.csv(sales_path,header=True,inferSchema=True,sep=",",comment="#",nullValue="na",modifiedAfter='2025-12-18',dateFormat="yyyy-dd-MM",ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True,quote='"',nanValue=-1 ,escape="\\")
df_sales.show(5)
display(df_sales)
#nan value works on float/double columns only not on numeric

###1. CSV Advanced Features - 
######Very Important - path: PathOrPaths, schema: Optional[Union[StructType, str]]=None, sep: Optional[str]=None,header: Optional[Union[bool, str]]=None, inferSchema: Optional[Union[bool, str]]=None, 
######Important - mode: Optional[str]=None, columnNameOfCorruptRecord: Optional[str]=None,  quote: Optional[str]=None, escape: Optional[str]=None, 
Not Important but good to know once - encoding: Optional[str]=None, comment: Optional[str]=None,ignoreLeadingWhiteSpace: Optional[Union[bool, str]]=None, ignoreTrailingWhiteSpace: Optional[Union[bool, str]]=None, nullValue: Optional[str]=None, nanValue: Optional[str]=None, positiveInf: Optional[str]=None, negativeInf: Optional[str]=None, dateFormat: Optional[str]=None, timestampFormat: Optional[str]=None, maxColumns: Optional[Union[int, str]]=None, maxCharsPerColumn: Optional[Union[int, str]]=None, maxMalformedLogPerPartition: Optional[Union[int, str]]=None,   multiLine: Optional[Union[bool, str]]=None, charToEscapeQuoteEscaping: Optional[str]=None, samplingRatio: Optional[Union[float, str]]=None, enforceSchema: Optional[Union[bool, str]]=None, emptyValue: Optional[str]=None, locale: Optional[str]=None, lineSep: Optional[str]=None, pathGlobFilter: Optional[Union[bool, str]]=None, recursiveFileLookup: Optional[Union[bool, str]]=None, modifiedBefore: Optional[Union[bool, str]]=None, modifiedAfter: Optional[Union[bool, str]]=None, unescapedQuoteHandling: Optional[str]=None) -> "DataFrame"

### A. Options for handling quotes & Escape

id,name,remarks
1,'Ramesh, K.P','Good performer'
2,'Manoj','Needs ~'special~' attention'

In [0]:
#When to go for quote: If the data is having delimiter in it..
#When to go for escape: If the data is having quote in it...
struct1="custid int,name string,age int,corrupt_record string"
df1=spark.read.schema(struct1).csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/malformed_data.txt",header=False,sep=',',mode='permissive',comment='#',columnNameOfCorruptRecord="corrupt_record",quote='"',escape="|")
df1.show(5)

### B. Comments, Multi line, leading and trailing whitespace handling, null and nan handling

In [0]:
from pyspark.sql.functions import *
struct1="custid int,name string,height float,joindt date,age string"
df1=spark.read.schema(struct1).csv("/Volumes/workspace/default/volumewd36/malformeddata2.txt",header=False,mode='permissive'
                                   ,multiLine=True,quote="'",ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True,
                                   nullValue='na',nanValue=-1,maxCharsPerColumn='100',modifiedAfter='2025-12-19',dateFormat="yyyy-dd-MM")
display(df1)

### C. Read modes in csv (Important feature)
If any data challenges (malformed data) such as format issue/column numbers (lesser/more than expected) issue etc.,
### There are 3 typical read modes and the default read mode is permissive.
##### 1. permissive — All fields are set to null and corrupted records are placed in a string column called _corrupt_record
##### 	2. dropMalformed — Drops all rows containing corrupt records.
##### 3. failFast — Fails when corrupt records are encountered.

In [0]:
#We learned about few important features mode, columnNameOfCorruptRecord, Quote, Comment
#Question - Corrupt_record column consume more memory because it capturing all the column values(incorrect) in one column. ? Useful for doing RCA (Root Cause Analysis/Debugging)
struct1="custid int,name string,age int,corrupt_record string"
df1=spark.read.schema(struct1).csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/malformeddata.txt",header=False,sep=',',mode='permissive',comment='#',columnNameOfCorruptRecord="corrupt_record",quote="'")
df1.show(10)
df1=spark.read.schema(struct1).csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/malformeddata.txt",header=False,sep=',',mode='dropMalformed',comment='#',columnNameOfCorruptRecord="corrupt_record",quote="'")
df1.show(10)
df1=spark.read.schema(struct1).csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/malformeddata.txt",header=False,sep=',',mode='failFast',comment='#',columnNameOfCorruptRecord="corrupt_record",quote="'")
df1.show(10)

####5. Reading data from other formats 

####1. Reading csv data

In [0]:
spark.read.options(inferSchema='true',sep='~',header='true').csv("/Volumes/ag_catalog/ag_practice/ag_volume/target/csvout").show(2)


###2. JSON Advanced Features - 
**Very Important** - path,schema,columnNameOfCorruptRecord,dateFormat,timestampFormat,multiLine,pathGlobFilter,recursiveFileLookup<br>
No header, No inferSchema, No sep in json...<br>
**Important** - primitivesAsString(don't do inferSchema), prefersDecimal, allowComments, allowUnquotedFieldNames, `allowSingleQuotes`, lineSep, samplingRatio, dropFieldIfAllNull, modifiedBefore, modifiedAfter, useUnsafeRow(This is performance optimization when the data is loaded into spark memory) <br>
**Not Important** (just try to know once for all) - allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, allowUnquotedControlChars, encoding, locale, allowNonNumericNumbers<br>%md
####2. Reading json data

In [0]:
spark.read.json("/Volumes/ag_catalog/ag_practice/ag_volume/target/jsonout").show(2)

In [0]:
#https://spark.apache.org/docs/latest/sql-data-sources-json.html#data-source-option
#primitivesAsString = inferSchema=False
dfjson1=spark.read.json("/Volumes/ag_catalog/ag_practice/ag_volume/target/jsonout",primitivesAsString=True)
dfjson1.printSchema()
#prefersDecimal
dfjson1=spark.read.json("/Volumes/ag_catalog/ag_practice/ag_volume/target/jsonout",prefersDecimal=True)
dfjson1.printSchema()

####3. Reading xml data

In [0]:
spark.read.xml("/Volumes/ag_catalog/ag_practice/ag_volume/target/xmlout",rowTag="cust").show(2)

###Reading serialized data (orc/parquet/delta)

###3. Serialized data Advanced Feature - orc, parquet/delta (very very important & we learn indepth)
- PathOrPaths
- **mergeSchema** - Important interview property (make it proactive/make it driven in the interview) SCHEMA EVOLUTION
- pathGlobFilter
- recursiveFileLookup
- modifiedBefore
- modifiedAfter
Problem statement:
Source is sending data in any way they want...
Day1/source1- 5 cols
Day2/source2 - 7 Cols

1. I am reading the dataframe in csv/json...
2. Writing into a orc/parquet format in a single location.
3. Reading data in a orc/parquet format using mergeSchema option.

In [0]:
#Case study: If the source (external) is sending data to us, if our consumer (Datascience/Dataanalytics) is directly communicated with our source system and asked them to propogate more/less attributes/features without the knowledge of the Dataengineering team? Here we have implement the strategy of Schema Evolution using option mergeSchema
#Story building for interview: We get product data from source which get evolved on a frequent basis for eg. product data originally sent without gendra, costprice, purchaseprice, profit/loss metrics, demant information...
#Steps to follow:
#1. Collect the data as it is from the source
#2. Convert into orc/parquet format and write to the target by appending the data on a regular interval
#3. Read the data from the target and do the schema evolution and get the evolved dataframe created...

In [0]:
spark.read.orc("/Volumes/ag_catalog/ag_practice/ag_volume/target/orcout",mergeSchema=True).show(2)
spark.read.parquet("/Volumes/ag_catalog/ag_practice/ag_volume/target/parquetout",mergeSchema=True).show(2)
spark.read.format("delta").load("/Volumes/ag_catalog/ag_practice/ag_volume/target/deltaout").show(2)

####5. Reading delta/hive table data

In [0]:
spark.table("ag_catalog.ag_practice.lh_custtbl").show(5)